Script para Unificar os arquivos em CSV

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, Table, Column, MetaData
from sqlalchemy.types import String
import os
import chardet

# Função para detectar a codificação de um arquivo
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# Função para limpar o nome das tabelas e colunas (transformando para maiúsculas)
def clean_name(name):
    return name.strip().replace('-', '_').replace(' ', '_').replace('.', '_').upper()

# Caminho dos arquivos CSV
csv_path_escolas = r"C:\Users\jvfal\OneDrive\Área de Trabalho\Teste Analista Jr\data_analytics_jr_test\Data\Escolas"
csv_files_escolas = [
    'escolas122018.csv', 'escolas122019.csv', 'escolas122020.csv', 
    'escolas122021.csv', 'escolas122022.csv', 'escolas122023.csv', 
    'escolas-dez-2010.csv', 'escolas-dez-2011.csv', 'escolas-dez-2012.csv',
    'escolas-dez-2013.csv', 'escolas-dez-2014.csv', 'escolas-dez-2015.csv',
    'escolasr34dez2017.csv', 'escolasr34.csv'
]
csv_path_alunos = r"C:\Users\jvfal\OneDrive\Área de Trabalho\Teste Analista Jr\data_analytics_jr_test\Data\Perfil dos educandos"
csv_files_alunos = [
    'idadeserieneeracadez17.csv', 'idadeserieneeracadez18.csv', 
    'idadeserieneeracadez19.csv', 'idadeserieneeracadez20.csv', 
    'idadeserieneeracadez21.csv', 'idadeserieneeracadez22.csv', 
    'idadeserieneeracadez23.csv', 'idadeserieneeraca-r33.csv'
]

# Conectando ao banco 
engine = create_engine('sqlite:///test_analytics.db', echo=True)
metadata = MetaData()

# Armazena o dicionário de dados
colunas_esperadas_alunos = pd.read_excel(r'C:\Users\jvfal\OneDrive\Área de Trabalho\Teste Analista Jr\data_analytics_jr_test\Data\Perfil dos educandos\dicionariopefileducando.xlsx')['CAMPO '].to_list()
colunas_esperadas_escolas = pd.read_excel(r'C:\Users\jvfal\OneDrive\Área de Trabalho\Teste Analista Jr\data_analytics_jr_test\Data\Escolas\dicionarioescolas.xlsx')['CAMPO'].to_list()

# Função para alinhar as colunas 
def alinhar_colunas(df, colunas_esperadas):
    for column_df in df.columns:
        if column_df not in colunas_esperadas:
            for coluna_esperada in colunas_esperadas:
                if column_df in coluna_esperada:
                    print(f'A coluna {column_df} é parecida com {coluna_esperada} e será renomeada como tal.')
                    df = df.rename({column_df:coluna_esperada}, axis=1)
                    break

    # Verifica se as colunas esperadas estão no dataframe
    for col in colunas_esperadas:
        if col not in df.columns:
            print(f'Adicionando a coluna {col}')
            df[col] = None 
    # Ordena as colunas conforme o esperado
    df = df[colunas_esperadas]  
    return df

# Função para limpar colunas indesejadas ou vazias
def limpar_colunas(df):
    # Remove colunas que são completamente vazias ou com nomes estranhos
    df = df.dropna(axis=1, how='all')  # Remove colunas completamente vazias
    df = df.loc[:, ~df.columns.str.contains('^Unnamed|^$', regex=True)]  # Remove colunas 'Unnamed' ou com nome vazio
    return df

# Função para combinar os arquivos CSV em um único DataFrame
def combine_csv(files, file_path, delimiter=';', categoria=''):
    combined_df = pd.DataFrame()
    colunas_esperadas = None  # Lista para armazenar as colunas esperadas

    for file in files:
        file_path_full = os.path.join(file_path, file)
        
        try:
            # Detecta a codificação do arquivo
            encoding = detect_encoding(file_path_full)
            print('*' * 30)
            print(f"Detectada codificação {encoding} para o arquivo {file}")
            
            # Lê o arquivo CSV
            df = pd.read_csv(file_path_full, delimiter=delimiter, encoding=encoding)
            print(f"Arquivo {file} carregado com sucesso. Informações: {df.shape[0]} linhas e {df.shape[1]} colunas")
            print(f"Primeiras linhas do arquivo {file}:\n{display(df.head(1))}")

            # Limpa o nome das colunas (transformando para maiúsculas)
            df.columns = [clean_name(col) for col in df.columns]

            # Limpa colunas indesejadas ou com nome estranho
            # df = limpar_colunas(df)

            if categoria == 'aluno':
                colunas_esperadas = colunas_esperadas_alunos
            elif categoria == 'escola':
                colunas_esperadas = colunas_esperadas_escolas

            # Alinha as colunas do DataFrame atual com as 'colunas_esperadas'
            df = alinhar_colunas(df, colunas_esperadas)

            print(f"Primeiras linhas do arquivo após tratamento:\n{display(df.head(1))}")
            print(f"{df.shape[0]} linhas e {df.shape[1]} colunas")

            # Remove colunas completamente vazias (ou com NaN)
            #df = df.dropna(axis=1, how='all')  # Remove colunas vazias ou com todos os valores NaN

            # Adiciona o DataFrame ao combinado
            combined_df = pd.concat([combined_df, df], ignore_index=True, sort=False)
            
        except Exception as e:
            print(f"Erro ao processar o arquivo {file}: {e}")
    
    return combined_df

def formatar_data(data):
    data = data
    meses = {
        'jan': '01',
        'fev': '02',
        'mar': '03',
        'abr': '04',
        'mai': '05',
        'jun': '06',
        'jul': '07',
        'ago': '08',
        'set': '09',
        'out': '10',
        'nov': '11',
        'dez': '12'
    }
    try:
        mes = str(data)[3:6]
        data = str(data)
        if mes in meses.keys():
            data_tratada = data.replace(mes, meses[mes]).replace('-', '/')
            return pd.to_datetime(data_tratada, format='%d/%m/%y').strftime('%d/%m/%Y')
        else:
            # Formato YYYYMMDD
            data_tratada = data[:10]
            return pd.to_datetime(data_tratada, format='%d/%m/%Y').strftime('%d/%m/%Y')
    except ValueError:
        #print(f"Erro ao formatar a data: {data}")
        return None

# Combina todos os arquivos de escolas e alunos
escolas_df = combine_csv(csv_files_escolas, csv_path_escolas, categoria='escola')
alunos_df = combine_csv(csv_files_alunos, csv_path_alunos, categoria='aluno')
escolas_df['DATABASE'] = escolas_df['DATABASE'].apply(formatar_data)
alunos_df['DATABASE'] = alunos_df['DATABASE'].apply(formatar_data)

alunos_df['ID_ALUNO'] = alunos_df.index
alunos_df = alunos_df[['ID_ALUNO', 'DRE', 'CODESC', 'TIPOESC', 'NOMESC', 'DISTRITO', 'SETOR', 'ANO',
    'REDE', 'MODAL', 'DESCSERIE', 'PERIODO', 'TURNO', 'DESCTURNO', 'SEXO', 'IDADE', 
    'NEE', 'RACA', 'QTDE', 'DATABASE']]
escolas_df['ID_ESCOLA'] = escolas_df.index
escolas_df = escolas_df[['ID_ESCOLA', 'DRE', 'CODESC', 'TIPOESC', 'NOMESC', 'NOMESCOF', 'CEU', 'DIRETORIA',
       'SUBPREF', 'ENDERECO', 'NUMERO', 'BAIRRO', 'CEP', 'TEL1', 'TEL2', 'FAX',
       'SITUACAO', 'CODDIST', 'DISTRITO', 'SETOR', 'CODINEP', 'CODCIE', 'EH',
       'FX_ETARIA', 'DT_CRIACAO', 'ATO_CRIACAO', 'DOM_CRIACAO', 'DT_INI_FUNC',
       'DT_INI_CONV', 'DT_AUTORIZA', 'DT_EXTINCAO', 'NOME_ANT', 'REDE',
       'LATITUDE', 'LONGITUDE', 'DATABASE']]

# Verifica as colunas após a remoção das duplicatas
print(f"Colunas após combinação em 'escolas': {escolas_df.columns}")
print(f"Colunas após combinação em 'alunos': {alunos_df.columns}")

# Função para criar a tabela a partir do DataFrame
def create_table_from_dataframe(df, table_name):
    columns = [Column(col, String) for col in df.columns]
    table = Table(table_name, metadata, *columns)

    # Cria a tabela no banco de dados
    metadata.create_all(engine)
    print(f"Tabela '{table_name}' criada com sucesso!")

# Cria a tabela 'escolas' no banco de dados
create_table_from_dataframe(escolas_df, 'escolas')

# Cria a tabela 'alunos' no banco de dados
create_table_from_dataframe(alunos_df, 'alunos')

# Função para inserir os dados no banco de dados usando o método `insert()` do SQLAlchemy
def insert_data_from_dataframe(df, table_name):
    with engine.connect() as conn:
        # Inserir os dados em massa utilizando o método insert() do SQLAlchemy
        df.to_sql(table_name, conn, if_exists='append', index=False)
        print(f"Dados inseridos na tabela '{table_name}' com sucesso!")

# Função para Inserir os dados na tabela 'escolas'
insert_data_from_dataframe(escolas_df, 'escolas')

# Função para Inserir os dados na tabela 'alunos'
insert_data_from_dataframe(alunos_df, 'alunos')

print("Processo concluído!")





******************************
Detectada codificação UTF-8-SIG para o arquivo escolas122018.csv
Arquivo escolas122018.csv carregado com sucesso. Informações: 4753 linhas e 34 colunas


,dre,codesc,tipoesc,nomesc,ceu,diretoria,subpref,endereco,numero,bairro,...,doc_criacao,dt_ini_conv,dt_ini_func,dt_autoriza,dt_extintao,nome_ant,rede,latitude,longitude,database
0,BT,191,EMEF,"ALIPIO CORREA NETO, PROF",NaN,DIRETORIA REGIONAL DE EDUCACAO BUTANTA,BUTANTA,Avenida JOAO CAIAFFA,140,JARDIM TABOAO,...,27/05/1988,NaN,02/10/1988,16/03/1991,NaN,"ALIPIO CORREA NETO, PROF.",DIR,-23.612.237,-46.749.888,30/12/2018


Primeiras linhas do arquivo escolas122018.csv:
None
Adicionando a coluna NOMESCOF
Adicionando a coluna CODCIE
Adicionando a coluna DOM_CRIACAO
Adicionando a coluna DT_EXTINCAO


,DRE,CODESC,TIPOESC,NOMESC,NOMESCOF,CEU,DIRETORIA,SUBPREF,ENDERECO,NUMERO,...,DOM_CRIACAO,DT_INI_FUNC,DT_INI_CONV,DT_AUTORIZA,DT_EXTINCAO,NOME_ANT,REDE,LATITUDE,LONGITUDE,DATABASE
0,BT,191,EMEF,"ALIPIO CORREA NETO, PROF",None,NaN,DIRETORIA REGIONAL DE EDUCACAO BUTANTA,BUTANTA,Avenida JOAO CAIAFFA,140,...,None,02/10/1988,NaN,16/03/1991,None,"ALIPIO CORREA NETO, PROF.",DIR,-23.612.237,-46.749.888,30/12/2018


Primeiras linhas do arquivo após tratamento:
None
4753 linhas e 35 colunas
******************************
Detectada codificação UTF-8-SIG para o arquivo escolas122019.csv
Arquivo escolas122019.csv carregado com sucesso. Informações: 4910 linhas e 34 colunas


...

Primeiras linhas do arquivo após tratamento:
None
169100 linhas e 19 colunas
******************************
Detectada codificação ISO-8859-1 para o arquivo idadeserieneeraca-r33.csv
Arquivo idadeserieneeraca-r33.csv carregado com sucesso. Informações: 374610 linhas e 21 colunas


,DRE,CODESC,TIPOESC,NOMESC,DISTRITO,SETEDU,ANO,REDE,MODAL,MODALID,...,DESCSERIE,PERIODO,TURNO,DESCTURNO,SEXO,IDADE,NEE,RACA,QTDE,DATABASE
0,BT,191,EMEF,"ALIPIO CORREA NETO, PROF.",VILA SONIA,9404,2016,DIR,ATCOMP,ATCOMP,...,JOGOS EDUCATIVO,DIURNO,3,Tarde,F,12,NAO POSSUI,PARDA,2,30/09/2016


Primeiras linhas do arquivo idadeserieneeraca-r33.csv:
None
Adicionando a coluna SETOR


,DRE,CODESC,TIPOESC,NOMESC,DISTRITO,SETOR,ANO,REDE,MODAL,DESCSERIE,PERIODO,TURNO,DESCTURNO,SEXO,IDADE,NEE,RACA,QTDE,DATABASE
0,BT,191,EMEF,"ALIPIO CORREA NETO, PROF.",VILA SONIA,None,2016,DIR,ATCOMP,JOGOS EDUCATIVO,DIURNO,3,Tarde,F,12,NAO POSSUI,PARDA,2,30/09/2016


Primeiras linhas do arquivo após tratamento:
None
374610 linhas e 19 colunas
Colunas após combinação em 'escolas': Index(['ID_ESCOLA', 'DRE', 'CODESC', 'TIPOESC', 'NOMESC', 'NOMESCOF', 'CEU',
       'DIRETORIA', 'SUBPREF', 'ENDERECO', 'NUMERO', 'BAIRRO', 'CEP', 'TEL1',
       'TEL2', 'FAX', 'SITUACAO', 'CODDIST', 'DISTRITO', 'SETOR', 'CODINEP',
       'CODCIE', 'EH', 'FX_ETARIA', 'DT_CRIACAO', 'ATO_CRIACAO', 'DOM_CRIACAO',
       'DT_INI_FUNC', 'DT_INI_CONV', 'DT_AUTORIZA', 'DT_EXTINCAO', 'NOME_ANT',
       'REDE', 'LATITUDE', 'LONGITUDE', 'DATABASE'],
      dtype='object')
Colunas após combinação em 'alunos': Index(['ID_ALUNO', 'DRE', 'CODESC', 'TIPOESC', 'NOMESC', 'DISTRITO', 'SETOR',
       'ANO', 'REDE', 'MODAL', 'DESCSERIE', 'PERIODO', 'TURNO', 'DESCTURNO',
       'SEXO', 'IDADE', 'NEE', 'RACA', 'QTDE', 'DATABASE'],
      dtype='object')
2024-11-30 22:26:40,461 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-30 22:26:40,462 INFO sqlalchemy.engine.Engine PRAGMA main.table_in